In [4]:
## Importing Libraries

import requests
import pickle
import pandas as pd
import numpy as np
from flask import Flask, request
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Importing Data

htr = pd.read_csv('houses_to_rent_v2.csv',na_values='-').fillna(0)

## Defining Variables

X = htr.drop(['rent amount (R$)','total (R$)'],axis=1)
y = htr['rent amount (R$)']

## One Hot Encoding

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['city','animal','furniture'])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
pickle.dump(ct,open('one_hot_enc.pkl','wb'))

### Feature Scaling

sc = StandardScaler()
X = sc.fit_transform(X)
pickle.dump(sc,open('stand_scaler.pkl','wb'))

## Defining training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Multiple Linear Regression Model

model = LinearRegression()
model.fit(X_train, y_train)


pickle.dump( model, open( 'model/house_model.pkl', 'wb' ) )

In [5]:
df = htr.drop(['rent amount (R$)','total (R$)'],axis=1).sample()
df_json = df.to_json(orient = 'records')

In [8]:
url = 'http://127.0.0.1:5000/predict'
data = df_json
header = {'Content-type':'application/json'}

r = requests.post(url=url,data=data,headers=header)

print(r.status_code)

200


In [9]:
pd.DataFrame(r.json(),columns=r.json()[0].keys())

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),property tax (R$),fire insurance (R$),prediction
0,São Paulo,52,2,2,1,5.0,not acept,furnished,903,300,51,4031.88651
